In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import csv

In [2]:
# load nasich + Honzovo dat a spojeni do velke tabulky, odstaneni radku, kde je delay = NaN
data_nase = pd.read_csv('out.c-cleaning.ALL_POSITIONS_NEW.csv', encoding='utf-8', low_memory=False)
data_Honza = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW.csv', encoding='utf-8', low_memory=False)
data_wkend = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW_WKEND.csv', encoding='utf-8', low_memory=False)
data_wkend_nase = pd.read_csv('out_clean_vikend_16_17.csv', encoding='utf-8', low_memory=False)

data = pd.concat([data_nase, data_Honza, data_wkend,data_wkend_nase], ignore_index = True)
data = data.dropna(axis=0,subset=['DELAY'],how='any')
data.head()

,UNICORN,TRIP_ID,START_TIME,HEADSIGN,DATE,BEARING,DELAY,DELAY_STOP_ARRIVAL,DELAY_STOP_DEPARTURE,IS_CANCELED,...,LAST_STOP_SEQUENCE,NEXT_STOP_ARRIVAL_TIME,NEXT_STOP_DEPARTURE_TIME,NEXT_STOP_ID,NEXT_STOP_SEQUENCE,ORIGIN_TIMESTAMP,SHAPE_DIST_TRAVELED,SPEED,LATITUDE,LONGITUDE
0,333_1093_200302_13_May,333_1093_200302,07:21:00,"Březová-Oleško,Oleško",13 May,187,-26.0,-29.0,NaN,False,...,27.0,"Wed, 13 May 2020 08:03:00 Z","Wed, 13 May 2020 08:03:00 Z",U1517Z1,28.0,"Wed, 13 May 2020 08:02:31 Z",18.9,40,49.90232,14.41034
1,335_353_191111_12_May,335_353_191111,18:28:00,"Praha,Budějovická",12 May,318,-132.0,NaN,-106.0,False,...,24.0,"Tue, 12 May 2020 19:22:00 Z","Tue, 12 May 2020 19:15:00 Z",U908Z2,25.0,"Tue, 12 May 2020 19:11:55 Z",23.8,44,50.02029,14.46425
2,335_353_191111_12_May,335_353_191111,18:28:00,"Praha,Budějovická",12 May,334,-147.0,NaN,-106.0,False,...,25.0,"Tue, 12 May 2020 19:22:00 Z","Tue, 12 May 2020 19:15:00 Z",U819Z2,26.0,"Tue, 12 May 2020 19:12:33 Z",24.2,0,50.02272,14.46133
3,335_353_191111_12_May,335_353_191111,18:28:00,"Praha,Budějovická",12 May,313,-31.0,NaN,3.0,False,...,26.0,"Tue, 12 May 2020 19:22:00 Z","Tue, 12 May 2020 19:17:00 Z",U488Z7,27.0,"Tue, 12 May 2020 19:15:45 Z",24.9,50,50.02833,14.45668
4,335_353_191111_12_May,335_353_191111,18:28:00,"Praha,Budějovická",12 May,358,-23.0,NaN,3.0,False,...,27.0,"Tue, 12 May 2020 19:22:00 Z","Tue, 12 May 2020 19:22:00 Z",U50Z84,28.0,"Tue, 12 May 2020 19:16:40 Z",25.2,11,50.03058,14.45512


In [3]:
# pridani sloupce s route_name
data['ROUTE_NAME'] = data['TRIP_ID'].str[:3]

In [4]:
# prevod sloupecku DATE na datovy format
data['DATE'] = data['DATE'] + " 2020"

In [5]:
data['DATE'] = pd.to_datetime(data['DATE'])

In [6]:
# vytvoreni noveho sloupce se dnem v tydnu
data['DAY_OF_WEEK'] = data['DATE'].dt.day_name()

In [7]:
# filtrace pouze linky 333, v ranni spicce, smer centrum -> novy mensi df
ranni_333 = data.loc[(data['HEADSIGN'] == 'Praha,Kačerov') & (data['ROUTE_NAME'] == '333'), ['UNICORN', 'TRIP_ID', 'START_TIME', 'HEADSIGN', 'DATE',
       'DELAY', 'DELAY_STOP_ARRIVAL', 'DELAY_STOP_DEPARTURE',
       'ORIGIN_TIMESTAMP', 'SHAPE_DIST_TRAVELED', 'SPEED', 'LATITUDE',
       'LONGITUDE']]

# pocet unikatnich linek 
len(ranni_333['UNICORN'].unique().tolist())

363

In [8]:
# soubor s jsonem se zastavkami z Dolnich Brezan na Kacerov
soubor = open('bus_stops_333_Brezany.json', encoding='utf-8')
data_stops = json.load(soubor)
soubor.close()

In [9]:
# vytvoreni tabulky z json souboru
info_bus_stops = pd.json_normalize(data_stops)

In [10]:
# dropnuti pro nas nepodstatnych sloupcu
info_stops_clean = info_bus_stops.drop(columns=['arrival_time_seconds','departure_time_seconds','drop_off_type','pickup_type','stop_headsign',
                                                'stop.properties.stop_code','stop.properties.stop_desc','stop.properties.stop_timezone','stop.properties.stop_url',
                                                'stop.properties.wheelchair_boarding','stop.properties.zone_id','stop.type','stop.geometry.type','stop.properties.level_id','stop.properties.location_type',
                                                'stop.properties.parent_station','stop.properties.platform_code','timepoint'])
# uprava sloupcu
newCols = ['ARRIVAL_TIME','DEPARTURE_TIME', 'SHAPE_DIST_TRAVELED', 'STOP_ID','STOP_SEQUENCE', 'TRIP_ID', 'GEOMETRY_COORDINATES','STOP_ID', 'STOP_NAME']
info_stops_clean.columns = newCols

In [28]:
# vybrani jednoho unicorna pro zjisteni all_positions -> ujetych km
unicorn_try = data[data['UNICORN'] == '333_1052_200302_06_May'].sort_values(by='SHAPE_DIST_TRAVELED')
len(unicorn_try)

97

In [12]:
# vybrani pouze sloupce s ujetymi km (-> z dataframu je serie)
stop_dist_333 = info_stops_clean['SHAPE_DIST_TRAVELED']
#stop_dist_333

In [13]:
bus_dist_333 = unicorn_try['SHAPE_DIST_TRAVELED']

In [14]:
# vytvoreni noveho sloupce pro nejblizsi pozici
info_stops_clean['NEJBLIZSI_POZICE'] = ''

In [15]:
i = 0
for distance in stop_dist_333:
    
    nejmensivzdalenost = 10000

    for position in bus_dist_333:

        tatonejmensivzdalenost = abs(distance - position)
        #print(tatonejmensivzdalenost)
        
        if tatonejmensivzdalenost < nejmensivzdalenost:
            
            nejmensivzdalenost = tatonejmensivzdalenost
            #print(nejmensivzdalenost)
            
            info_stops_clean.loc[i,'NEJBLIZSI_POZICE'] = position
    i += 1

In [16]:
info_stops_clean

,ARRIVAL_TIME,DEPARTURE_TIME,SHAPE_DIST_TRAVELED,STOP_ID,STOP_SEQUENCE,TRIP_ID,GEOMETRY_COORDINATES,STOP_ID,STOP_NAME,NEJBLIZSI_POZICE
0,10:32:00,10:32:00,0.00000,U1503Z2,1,333_566_200409,"[14.45708, 49.96319]",U1503Z2,"Dolní Břežany,Obecní úřad",0
1,10:33:00,10:33:00,0.15836,U1502Z2,2,333_566_200409,"[14.45889, 49.96373]",U1502Z2,"Dolní Břežany,Náměstí",0
2,10:35:00,10:35:00,0.77992,U1501Z2,3,333_566_200409,"[14.46199, 49.96894]",U1501Z2,"Dolní Břežany,Na kopečku",0.9
3,10:39:00,10:39:00,3.53721,U545Z2,4,333_566_200409,"[14.46853, 49.99163]",U545Z2,Písnice,3.4
4,10:40:00,10:40:00,3.91690,U546Z2,5,333_566_200409,"[14.46768, 49.99496]",U546Z2,Ke Březině,4.1
5,10:41:00,10:41:00,4.11683,U871Z2,6,333_566_200409,"[14.46765, 49.99676]",U871Z2,Lipovická,4.1
6,10:42:00,10:42:00,4.61837,U369Z2,7,333_566_200409,"[14.4666, 50.0012]",U369Z2,Sídliště Písnice,4.7
7,10:43:00,10:43:00,5.09108,U783Z2,8,333_566_200409,"[14.46417, 50.00515]",U783Z2,U Libušské sokolovny,5.1
8,10:44:00,10:44:00,5.46109,U335Z2,9,333_566_200409,"[14.46275, 50.00834]",U335Z2,Libuš,5.3
9,10:45:00,10:45:00,5.77175,U804Z2,10,333_566_200409,"[14.46015, 50.01056]",U804Z2,U Zvoničky,5.8


In [17]:
# soubor s jsonem se zastavkami z Brezove na Kacerov
soubor1 = open('bus_stops_333_Brezova.json', encoding='utf-8')
data_stops1 = json.load(soubor1)
soubor1.close()

In [18]:
# vytvoreni tabulky z json souboru
info_bus_stops1 = pd.json_normalize(data_stops1)

In [24]:
# dropnuti pro nas nepodstatnych sloupcu
info_stops_clean1 = info_bus_stops1.drop(columns=['arrival_time_seconds','departure_time_seconds','drop_off_type','pickup_type','stop_headsign',
                                                'stop.properties.stop_code','stop.properties.stop_desc','stop.properties.stop_timezone','stop.properties.stop_url',
                                                'stop.properties.wheelchair_boarding','stop.properties.zone_id','stop.type','stop.geometry.type','stop.properties.level_id','stop.properties.location_type',
                                                'stop.properties.parent_station','stop.properties.platform_code','timepoint'])
# uprava sloupcu
newCols = ['ARRIVAL_TIME','DEPARTURE_TIME', 'SHAPE_DIST_TRAVELED', 'STOP_ID','STOP_SEQUENCE', 'TRIP_ID', 'GEOMETRY_COORDINATES','STOP_ID', 'STOP_NAME']
info_stops_clean1.columns = newCols

In [38]:
info_stops_clean1.sort_values(by='STOP_SEQUENCE').head()

,ARRIVAL_TIME,DEPARTURE_TIME,SHAPE_DIST_TRAVELED,STOP_ID,STOP_SEQUENCE,TRIP_ID,GEOMETRY_COORDINATES,STOP_ID,STOP_NAME
1,14:02:00,14:02:00,0.00000,U1517Z2,1,333_1066_200302,"[14.41051, 49.902]",U1517Z2,"Březová-Oleško,Oleško"
2,14:03:00,14:03:00,0.75929,U1516Z2,2,333_1066_200302,"[14.41319, 49.90846]",U1516Z2,"Březová-Oleško,U Topolů"
6,14:04:00,14:04:00,1.16044,U9551Z2,3,333_1066_200302,"[14.41437, 49.91185]",U9551Z2,"Březová-Oleško,Školka"
7,14:05:00,14:05:00,1.63421,U1515Z2,4,333_1066_200302,"[14.41559, 49.91507]",U1515Z2,"Březová-Oleško,Březová"
8,14:07:00,14:07:00,2.58248,U1514Z2,5,333_1066_200302,"[14.41984, 49.92087]",U1514Z2,"Zvole,Černíky"
